In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [ ]:
### CREATE AZURE SEARCH INDEX [UPDATED FOR AZURE SEARCH SDK VERSION 11.6.0b5]

from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    HnswParameters,
    VectorSearchAlgorithmMetric,
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticSearch,
    SemanticPrioritizedFields,
    SemanticField,
    SearchIndex
)

# VARIABLES - AZURE SEARCH

# INSTANTIATE THE AZURE SEARCH INDEX CLIENT AND DEFINE AZURE SEACH SCHEMA
fields = [
    SearchField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchField(name="fileurl", type=SearchFieldDataType.String, filterable=True),
    SearchField(name="section_id", type=SearchFieldDataType.String, filterable=True),
    SearchField(name="section_name", type=SearchFieldDataType.String, filterable=True),
    SearchField(name="section_description", type=SearchFieldDataType.String),
    SearchField(name="section_chunk_id", type=SearchFieldDataType.Int32, filterable=True, sortable=True),
    SearchField(name="content", type=SearchFieldDataType.String),
    SearchField(name="content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, vector_search_dimensions=1536, vector_search_profile_name="ec_hnsw_profile_01"),
]

# VECTOR SEARCH CONFIGURATION
vector_search = VectorSearch(
    algorithms=[HnswAlgorithmConfiguration(
        name="ec_hnsw_algorithm_01",
        parameters=HnswParameters(
            m=4,
            ef_construction=400,
            ef_search=500,
            metric=VectorSearchAlgorithmMetric.COSINE))],
    profiles=[VectorSearchProfile(
        name="ec_hnsw_profile_01",
        algorithm_configuration_name="ec_hnsw_algorithm_01")])

# SEMANTIC SEARCH CONFIGURATION
semantic_config = SemanticConfiguration(
    name="ec_semantic_config_01",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="section_name"),
        content_fields=[SemanticField(field_name="content"), SemanticField(field_name="section_description")]))
semantic_search = SemanticSearch(configurations=[semantic_config])  

# CREATE THE AZURE SEARCH INDEX [INJECT FIELDS, VECTOR CONFIG, SEMANTIC CONFIG]
azure_search_index_client = SearchIndexClient(endpoint=os.environ["AZURE_COGNITIVE_SEARCH_ENDPOINT"], 
            credential=AzureKeyCredential(os.environ["AZURE_COGNITIVE_SEARCH_KEY"]))
index = SearchIndex(name=os.getenv("INDEX_NAME"), fields=fields, vector_search=vector_search, semantic_search=semantic_search)  
result = azure_search_index_client.create_or_update_index(index)  
print(f"{result.name} created")

excessive-casualty-index created
